This Jupyter notebook demonstrates use of the simulator to compute symbolic or numeric expectation values. See [`main.py`](main.py) for the equivalent code in a plain Python file.

## Imports

The PTM simulator maintains a weighted sum of Pauli strings, which we will evolve using symbolically parameterised operators.

In [1]:
from source . ensemble import WeightedSumOfPauliStrings
from source . symbols import Symb, Param, Prob
from source . operators import H, Rx, Ry, Rz, U, Damp, Deph, Depol, Kraus, PauliGadget

We'll also later repurpose some of the simulator's bitwise algebra for pretty printing.

In [2]:
from source . twiddles import getStringOfPauliInd

Finally, to specify arbitrary symbolic matrices, we will use some of [SymPy](https://docs.sympy.org/latest/index.html)'s symbolic math overloads.

In [3]:
from sympy import sqrt

## Preparing an initial state

Our initial state (a weighted sum of Pauli strings) is usually an operator of interest of which we later wish to compute the expectation value, expressed in the Pauli basis. Let's begin with $X_0 - 2 Y_2Z_0$.

In [4]:
state = WeightedSumOfPauliStrings()
state.addPauliString('X', 1)
state.addPauliString('YIZ', -2)

We can also use generic symbols as the initial weights:

In [5]:
a = Symb('a')
state.addPauliString('ZZZ', a)

or symbolic expressions:

In [6]:
b = Symb('b')
state.addPauliString('ZXY', sqrt(a)+b**2)

Because they are `Symb` instances, symbols `a` and `b` are treated as generic complex variables. We could also have specified them as `Param` (to constrain them to be real-valued), or as `Prob` (to constrain them to be probabilities, i.e. in `[0,1]`).

## Specifying a circuit

File [`operators.py`](source/operators.py) defines a few canonical operators

In [7]:
circ = [H(0), H(1), H(2)]

some of which can be parameterised

In [8]:
c = Param('c')
d = Param('d')
circ += [Rx(0, c), Ry(1, d)]

and include control qubits

In [9]:
e = Param('e')
circ += [Rz(2, e, ctrls=[0]), H(0, ctrls=[1,2])]

We also support any-target [Pauli gadgets](https://arxiv.org/abs/1906.01734)

In [10]:
f = Param('f')
circ += [PauliGadget([1,2], 'XY', f)]

and canonical error channels

In [11]:
g = Prob('g')
h = Prob('h')
circ += [Damp(0, g), Depol([0,1], h)]

For convenience, we can take a peek at the mathematics behind these operators.

In [12]:
H(0).getMatrix()

array([[sqrt(2)/2, sqrt(2)/2],
       [sqrt(2)/2, -sqrt(2)/2]], dtype=object)

In [13]:
Rx(0,c).getSuperOperator()

array([[cos(c/2)**2, -I*sin(c)/2, I*sin(c)/2, sin(c/2)**2],
       [-I*sin(c)/2, cos(c/2)**2, sin(c/2)**2, I*sin(c)/2],
       [I*sin(c)/2, sin(c/2)**2, cos(c/2)**2, -I*sin(c)/2],
       [sin(c/2)**2, I*sin(c)/2, -I*sin(c)/2, cos(c/2)**2]], dtype=object)

In [14]:
Rz(0,d).getPauliTransferMatrix()

array([[1, 0, 0, 0],
       [0, cos(d), -sin(d), 0],
       [0, sin(d), cos(d), 0],
       [0, 0, 0, 1]], dtype=object)

In [15]:
Damp(0,g).getPauliTransferMap()

			>>> evaluating and caching PTM of Damp1(real=True_nonnegative=True_probability=True)


[{0: 1.00000000000000, 3: g}, {1: sqrt(1 - g)}, {2: sqrt(1 - g)}, {3: 1 - g}]

## Applying a circuit

Let's apply the circuit so far

In [16]:
for op in circ:
    print('applying', op)
    state.applyOperator(op)

applying H[0]
			>>> evaluating and caching PTM of H1
applying H[1]
applying H[2]
applying Rx[0](c)
			>>> evaluating and caching PTM of Rx1(real=True_probability=False)
applying Ry[1](d)
			>>> evaluating and caching PTM of Ry1(real=True_probability=False)
applying C[0]Rz[2](e)
			>>> evaluating and caching PTM of C1Rz1(real=True_probability=False)
applying C[1,2]H[0]
			>>> evaluating and caching PTM of C2H1
applying PauliGadget[1,2][YX](f)
applying Damp[0](g)
applying Depol[0,1](h)
			>>> evaluating and caching PTM of Depol2(real=True_nonnegative=True_probability=True)


We have set `_LOG_CACHE=True` in [`paulitransfer.py`](source/paulitransfer.py) in order to see when an operator's Pauli transfer matrix was computed.

We can peek at the weight of one of the Pauli strings produced so far

In [17]:
state.getWeight('YYZ')

(1 - 16*h/15)*(g*(-0.25*sqrt(2)*a*cos(d)*cos(e/2) + (1/2 - cos(e)/2)*(-sqrt(a) - b**2)*sin(c)*sin(d)/2 - sqrt(2)*(-sqrt(a) - b**2)*(cos(e)/2 + 1/2)*sin(c)*sin(d)/4)*cos(f) + (1 - g)*(-1.0*(-0.25*sin(c)*sin(e/2) + 0.25*cos(c))*sin(f) + (-sqrt(2)*(1/2 - cos(e)/2)*(-sqrt(a) - b**2)*sin(c)*sin(d)/4 + (-sqrt(a) - b**2)*(cos(e)/2 + 1/2)*sin(c)*sin(d)/2 + sqrt(2)*(-sqrt(a) - b**2)*sin(d)*sin(e/2)*cos(c)/4)*cos(f)))

The result is a first-class [SymPy](https://www.sympy.org/en/index.html) [expression](https://docs.sympy.org/latest/tutorials/intro-tutorial/manipulation.html), so we can use the full API for subsequent calculation.

In [18]:
60 * sqrt(state.getWeight('YYZ')).simplify()

sqrt(30)*sqrt(16*g*h*(2.0*sqrt(2)*a*cos(d)*cos(e/2) + 2*(1 - cos(e))*(sqrt(a) + b**2)*sin(c)*sin(d) - sqrt(2)*(sqrt(a) + b**2)*(cos(e) + 1)*sin(c)*sin(d))*cos(f) - 15*g*(2.0*sqrt(2)*a*cos(d)*cos(e/2) + 2*(1 - cos(e))*(sqrt(a) + b**2)*sin(c)*sin(d) - sqrt(2)*(sqrt(a) + b**2)*(cos(e) + 1)*sin(c)*sin(d))*cos(f) - 16*h*(g - 1)*(-(sqrt(a) + b**2)*(sqrt(2)*(1 - cos(e))*sin(c) - 2*(cos(e) + 1)*sin(c) - 2*sqrt(2)*sin(e/2)*cos(c))*sin(d)*cos(f) + 2.0*(-sin(c)*sin(e/2) + cos(c))*sin(f)) + 15*(g - 1)*(-(sqrt(a) + b**2)*(sqrt(2)*(1 - cos(e))*sin(c) - 2*(cos(e) + 1)*sin(c) - 2*sqrt(2)*sin(e/2)*cos(c))*sin(d)*cos(f) + 2.0*(-sin(c)*sin(e/2) + cos(c))*sin(f)))

Because we have not substituted any values for our symbols `a ... h`, they are kept symbolic in the Pauli string weights. 

Calling `getWeight()` above triggered a heirarchal symbolic evaluation which we now clear in order to apply additional gates

In [19]:
state.clearWeights()
num = len(circ)

## Specifying general operators

We can describe general, multi-parameter operators in terms of their complex, symbolic matrices.

In [20]:
j = Symb('j')
k = Symb('k')

matr = (
  (0, 0, 1j, 0),
  (0, j+k, 0, 0),
  (j, 0, 0, j**2),
  (0, 0, 0, 0))

circ += [U([1,2], matr)]
circ += [U([0,2], matr, ctrls=[1])]

Note that `matr` above is not unitary for general `j`,`k`. Unitarity it is _not_ required by our PTM simulation, but it will result in our Pauli strings having _complex_ weights at later evalution

Even Kraus maps can be general:

In [21]:
kraus1 = ((0, j), (k, 0))
kraus2 = ((j+k, 0), (k**2, sqrt(j)))
krauses = (kraus1, kraus2)
circ += [Kraus(0, krauses)]
circ += [Kraus([1,2], (matr,))]

Let's also apply these additional operators

In [22]:
for op in circ[num:]:
    print('applying', op)
    state.applyOperator(op)

applying U[1,2](j,k)
			>>> evaluating and caching PTM of U2(4710293696)
applying C[1]U[0,2](j,k)
			>>> evaluating and caching PTM of C1U2(4710293696)
applying Kraus[0](j,k)
			>>> evaluating and caching PTM of Kraus1(4711162560)
applying Kraus[1,2](j,k)
			>>> evaluating and caching PTM of Kraus2(4710961536)


## Caching

When an unrecognised operator was first applied to our ensemble, its PTM was computed and the result was _cached_ to avoid future recomputation. As such, re-applying our circuit is now much faster:

In [23]:
num = len(circ)
circ *= 3
for op in circ[num:]:
    print('faster applying', op)
    state.applyOperator(op)

faster applying H[0]
faster applying H[1]
faster applying H[2]
faster applying Rx[0](c)
faster applying Ry[1](d)
faster applying C[0]Rz[2](e)
faster applying C[1,2]H[0]
faster applying PauliGadget[1,2][YX](f)
faster applying Damp[0](g)
faster applying Depol[0,1](h)
faster applying U[1,2](j,k)
faster applying C[1]U[0,2](j,k)
faster applying Kraus[0](j,k)
faster applying Kraus[1,2](j,k)
faster applying H[0]
faster applying H[1]
faster applying H[2]
faster applying Rx[0](c)
faster applying Ry[1](d)
faster applying C[0]Rz[2](e)
faster applying C[1,2]H[0]
faster applying PauliGadget[1,2][YX](f)
faster applying Damp[0](g)
faster applying Depol[0,1](h)
faster applying U[1,2](j,k)
faster applying C[1]U[0,2](j,k)
faster applying Kraus[0](j,k)
faster applying Kraus[1,2](j,k)


The cached PTMs are agnostic to target qubits and same-domain symbols.

In [24]:
num = len(circ)
circ += [
    Rz(2, c, ctrls=[0]),
    Rz(1, d, ctrls=[2]), 
    Rz(0, e, ctrls=[1]),
    Rz(0, f, ctrls=[2]), 
    H(2, ctrls=[0,1]),
    H(1, ctrls=[2,0]),
    Kraus(0, krauses),
    Kraus(1, krauses),
    Kraus(2, krauses),
    U([0,1], matr, ctrls=[2]),
    U([2,0], matr, ctrls=[1]),
    U([2,1], matr, ctrls=[0])
]

for op in circ[num:]:
    print('still faster applying', op)
    state.applyOperator(op)

still faster applying C[0]Rz[2](c)
still faster applying C[2]Rz[1](d)
still faster applying C[1]Rz[0](e)
still faster applying C[2]Rz[0](f)
still faster applying C[0,1]H[2]
still faster applying C[2,0]H[1]
still faster applying Kraus[0](j,k)
still faster applying Kraus[1](j,k)
still faster applying Kraus[2](j,k)
still faster applying C[2]U[0,1](j,k)
still faster applying C[1]U[2,0](j,k)
still faster applying C[0]U[2,1](j,k)


## Specifying non-CPTP operators

We saw that `U` and `Kraus` operators (specified element-wise) could be non-CPTP. So too can every canonical parameterised operator, by passing a permittedly-complex symbol.

In [25]:
l = Symb('l')
g1 = Ry(0, c) # real
g2 = Ry(1, l) # complex

for op in [g1, g2]:
    print(op, ':\n', op.getSuperOperator(), '\n')

Ry[0](c) :
 [[cos(c/2)**2 -sin(c)/2 -sin(c)/2 sin(c/2)**2]
 [sin(c)/2 cos(c/2)**2 -sin(c/2)**2 -sin(c)/2]
 [sin(c)/2 -sin(c/2)**2 cos(c/2)**2 -sin(c)/2]
 [sin(c/2)**2 sin(c)/2 sin(c)/2 cos(c/2)**2]] 

Ry[1](l) :
 [[cos(l/2)*cos(conjugate(l)/2) -sin(l/2)*cos(conjugate(l)/2)
  -sin(conjugate(l)/2)*cos(l/2) sin(l/2)*sin(conjugate(l)/2)]
 [sin(l/2)*cos(conjugate(l)/2) cos(l/2)*cos(conjugate(l)/2)
  -sin(l/2)*sin(conjugate(l)/2) -sin(conjugate(l)/2)*cos(l/2)]
 [sin(conjugate(l)/2)*cos(l/2) -sin(l/2)*sin(conjugate(l)/2)
  cos(l/2)*cos(conjugate(l)/2) -sin(l/2)*cos(conjugate(l)/2)]
 [sin(l/2)*sin(conjugate(l)/2) sin(conjugate(l)/2)*cos(l/2)
  sin(l/2)*cos(conjugate(l)/2) cos(l/2)*cos(conjugate(l)/2)]] 



In [26]:
m = Symb('m')
g3 = Damp(0, g) # probability
g4 = Damp(1, m) # complex

for op in [g3, g4]:
    print(op, ':\n', op.getSuperOperator(), '\n')

Damp[0](g) :
 [[1 0 0 g]
 [0 sqrt(1 - g) 0 0]
 [0 0 sqrt(1 - g) 0]
 [0 0 0 1 - g]] 

Damp[1](m) :
 [[1 0 0 sqrt(m)*conjugate(sqrt(m))]
 [0 sqrt(1 - m) 0 0]
 [0 0 conjugate(sqrt(1 - m)) 0]
 [0 0 0 sqrt(1 - m)*conjugate(sqrt(1 - m))]] 



Let's apply these non-CPTP operators to our ensemble. Note their PTMs are computed and cached afresh, because the existing cached ones were simplified by assuming CPTP.

In [27]:
circ += [g2, g4]
for op in circ[-2:]:
    print('applying non-CPTP', op)
    state.applyOperator(op)

applying non-CPTP Ry[1](l)
			>>> evaluating and caching PTM of Ry1(probability=False)
applying non-CPTP Damp[1](m)
			>>> evaluating and caching PTM of Damp1(probability=False)


## Susbtituting parameters

By now, our Pauli ensemble is sufficiently complicated that evaluating a Pauli string weight _symbolically_, as we did earlier, would be infeasible. So instead, we specify numerical values for the symbols:

In [28]:
values = {
    a: 1+1j, b: -1-0.5j,     # Symb
    c: 1, d: 2, e: 3, f: 4,  # Param
    g: .1, h:.1,             # Prob
    j: 1j, k: 2j, l: 1, m:2  # Symb
}

We bind these substitutions to the ensemble, so that they are leveraged during subsequent weight calculation in a hierarchical fashion.

In [29]:
state.setParams(values)

Now we can compute _every_ weight numerically.

In [30]:
numQubits = state.getNumQubits()
numStates = state.getNumPossibleStrings()

for ind in range(numStates):
    weight = state.getWeight(ind)
    print(getStringOfPauliInd(ind, numQubits), '=', weight)

III = -491277695440.553 - 300276209764.428*I
IIX = 49164014288.627 + 29641894531.5694*I
IIY = -104914096181.537 - 64345510766.092*I
IIZ = 216540643042.074 + 131115483617.428*I
IXI = -15134980887.631 - 9315807267.18915*I
IXX = -105517791501.864 - 65083395544.2425*I
IXY = -69315800081.2531 - 41176322976.1162*I
IXZ = -13676196569.0263 - 8435867204.64878*I
IYI = -160276585904.166 - 97506319058.6129*I
IYX = 31632863263.9426 + 19023269973.0902*I
IYY = -69609524936.6136 - 42655212403.3244*I
IYZ = 241513856542.033 + 147351981910.062*I
IZI = -296390049475.202 - 180904380673.64*I
IZX = -8185202961.57219 - 4608797576.77921*I
IZY = -11875556103.7221 - 7058866232.37562*I
IZZ = 187031743326.754 + 113712077223.425*I
XII = 14315601524.8193 + 9152132718.77937*I
XIX = -39236910843.816 - 23947737438.9233*I
XIY = 1741850089.34559 + 1303821926.29824*I
XIZ = -20215681275.5768 - 11322921423.1031*I
XXI = -5694262261.3466 - 3659761212.64569*I
XXX = -20938983078.9368 - 12232495330.3379*I
XXY = 5685754024.49656 

## Partially substituting parameters

We do not need to give _every_ symbol a value. We can keep some generic, so that the calculated weights contain only specific symbols.

Here are some operators with new parameters

In [31]:
n = Param('n')
p = Prob('p')
q = Symb('q')

num = len(circ)
circ += [
    PauliGadget([0,1,2], 'XYZ', n),
    Deph([0,1], p),
    Ry([2], q, ctrls=[1])
]

Let's apply these to our ensemble, but without giving them explicit values in the substitution.

In [32]:
state.clearWeights()

for op in circ[num:]:
    print('applying', op)
    state.applyOperator(op)

state.setParams(values) # excludes n,p,q

applying PauliGadget[0,1,2][ZYX](n)
applying Deph[0,1](p)
			>>> evaluating and caching PTM of Deph2(real=True_nonnegative=True_probability=True)
applying C[1]Ry[2](q)
			>>> evaluating and caching PTM of C1Ry1(probability=False)


Pauli string weights will now be parameterised only in terms of `n`, `p` and `q`: 

In [33]:
state.getWeight('XYZ')

(1 - 4*p/3)*(1.0*(-104914096181.537 - 64345510766.092*I)*sin(n) + (-240429718647.096 - 146697490379.496*I)*cos(n))*(sin(q/2)/2 + sin(conjugate(q)/2)/2) + I*(1 - 4*p/3)*(1.0*(9112926819.26284 + 5553976453.00949*I)*sin(n) + (-3301514028.42751 - 1505808366.01058*I)*cos(n))*(-cos(q/2) + cos(conjugate(q)/2))/2 + I*(1 - 4*p/3)*(13832197413.7891 + 8528244825.12275*I)*(-sin(q/2) + sin(conjugate(q)/2))/2 + (-11035395121.9849 - 6808217348.04706*I)*(1 - 4*p/3)*(cos(q/2)/2 + cos(conjugate(q)/2)/2)

This output is still a [SymPy](https://www.sympy.org/en/index.html) [expression](https://docs.sympy.org/latest/tutorials/intro-tutorial/manipulation.html), which we can continue to symbolically process.

In [34]:
state.getWeight('XYZ').simplify()

(4*p - 3)*(I*((9112926819.26284 + 5553976453.00949*I)*sin(n) - (3301514028.42751 + 1505808366.01058*I)*cos(n))*(cos(q/2) - cos(conjugate(q)/2)) + ((104914096181.537 + 64345510766.092*I)*sin(n) + (240429718647.096 + 146697490379.496*I)*cos(n))*(sin(q/2) + sin(conjugate(q)/2)) + I*(13832197413.7891 + 8528244825.12275*I)*(sin(q/2) - sin(conjugate(q)/2)) + (11035395121.9849 + 6808217348.04706*I)*(cos(q/2) + cos(conjugate(q)/2)))/6

Let's restore the weights to be fully numerical:

In [35]:
values.update({n: -.1, p: .2, q: -1-1j})
state.setParams(values)

## Computing the expectation value

Recall that our initial ensemble was

$$
\hat{O} = \hat{X}_0 - 2 \, \hat{Y}_2 \hat{Z}_0 + a\, \hat{Z}_2 \hat{Z}_1 \hat{Z}_0 + b \, \hat{Z}_2 \hat{X}_1 \hat{Y}_0
$$

This is the operator of which we wish to compute the expectation value, which is merely the sum of a specific subset of the final Pauli string weights; those overlapping our initial state, $\ket{\ket{0}}$.

In [36]:
expec = state.getZeroOverlap()
expec

-33295482650.7938 - 20408816166.0742*I

## Truncating branches

In our tiny three-qubit example, propogating the Pauli ensemble was negligibly costly; we spent most of the time computing PTMs. In a sensible regime however, ensemble propogation dominates runtime due to the very large and quickly growing number of Pauli strings.

To keep this number tractable and propogation fast, we can _truncate_ the ensemble during simulation. Let's repeat our simulation, discarding Pauli strings of `<1%` "lineage", after every four operators applied.

In [37]:
state.clearOperators()

for i, op in enumerate(circ):
    print('applying', op)
    state.applyOperator(op)

    if not (i+1)%4:
        print('\ttruncating...')
        state.truncateByFracLineage(.01)

applying H[0]
applying H[1]
applying H[2]
applying Rx[0](c)
	truncating...
applying Ry[1](d)
applying C[0]Rz[2](e)
applying C[1,2]H[0]
applying PauliGadget[1,2][YX](f)
	truncating...
applying Damp[0](g)
applying Depol[0,1](h)
applying U[1,2](j,k)
applying C[1]U[0,2](j,k)
	truncating...
applying Kraus[0](j,k)
applying Kraus[1,2](j,k)
applying H[0]
applying H[1]
	truncating...
applying H[2]
applying Rx[0](c)
applying Ry[1](d)
applying C[0]Rz[2](e)
	truncating...
applying C[1,2]H[0]
applying PauliGadget[1,2][YX](f)
applying Damp[0](g)
applying Depol[0,1](h)
	truncating...
applying U[1,2](j,k)
applying C[1]U[0,2](j,k)
applying Kraus[0](j,k)
applying Kraus[1,2](j,k)
	truncating...
applying H[0]
applying H[1]
applying H[2]
applying Rx[0](c)
	truncating...
applying Ry[1](d)
applying C[0]Rz[2](e)
applying C[1,2]H[0]
applying PauliGadget[1,2][YX](f)
	truncating...
applying Damp[0](g)
applying Depol[0,1](h)
applying U[1,2](j,k)
applying C[1]U[0,2](j,k)
	truncating...
applying Kraus[0](j,k)
apply

Discarding Pauli strings introduces error into our expectation value; the output of `getZeroOverlap` is now an _approximation_.

In [38]:
trunced = state.getZeroOverlap()
trunced

-2628420767.00706 + 1457091434.8932*I

For such a tiny system ($3$ qubits, with a total of $64$ possible Pauli states), truncation which discards any of the $8$ overlapping Pauli strings naturally causes a catastrophic relative error:

In [39]:
abs((expec-trunced)/expec)*100

96.4445059733876

But you get the idea `¯\_(ツ)_/¯`